In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./1.category.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Url'] = [url_1 if url_2 == '-' else url_2 for url_1, url_2 in zip(input_['1st Category Url'].tolist(), input_['2nd Category Url'].tolist())]

input_['1st Category No'] = input_['1st Category No'].astype(int)
input_['2nd Category No'] = ['-' if _ == '-' else int(_) for _ in input_['2nd Category No'].tolist()]

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            b = 0
            while True:
                b += 1

                try:
                    resp = requests.get(input_.loc[a, 'Url'],
                                        headers=get_header(ua=False),
                                        proxies=get_proxy(),
                                        timeout=(10, 10))

                    if resp.status_code == 200:
                        break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            # = = = = = = = = = = = = = = =

            count = int(html.xpath('//h1[@class="srp-controls__count-heading"]/text()')[0].strip().removesuffix('result').removesuffix('results').strip().removesuffix('+').replace(',', '').replace('.', ''))

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame([{'1st Category No': input_.loc[a, '1st Category No'],
                                     '1st Category Type': input_.loc[a, '1st Category Type'],
                                     '1st Category': input_.loc[a, '1st Category'],
                                     '1st Category Url': input_.loc[a, '1st Category Url'],
                                     '2nd Category No': input_.loc[a, '2nd Category No'],
                                     '2nd Category Type': input_.loc[a, '2nd Category Type'],
                                     '2nd Category': input_.loc[a, '2nd Category'],
                                     '2nd Category Url': input_.loc[a, '2nd Category Url'],
                                     'SKU Count': count,
                                     'Overlimit': False if count < 10_000 else True}])
            
            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([{'1st Category No': input_.loc[a, '1st Category No'],
                                     '1st Category Type': input_.loc[a, '1st Category Type'],
                                     '1st Category': input_.loc[a, '1st Category'],
                                     '1st Category Url': input_.loc[a, '1st Category Url'],
                                     '2nd Category No': input_.loc[a, '2nd Category No'],
                                     '2nd Category Type': input_.loc[a, '2nd Category Type'],
                                     '2nd Category': input_.loc[a, '2nd Category'],
                                     '2nd Category Url': input_.loc[a, '2nd Category Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, '1st Category No']}.{input_.loc[a, '1st Category']} > {input_.loc[a, '2nd Category No']}.{input_.loc[a, '2nd Category']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['1st Category No', '2nd Category No'],
                                                ascending=[True, True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./2.count_1-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['1st Category No', '2nd Category No'],
                                            ascending=[True, True],
                                            ignore_index=True)
    output_error.to_excel('./count_1_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：180

[状态：ok，尝试次数：1] - 1.Lighting > 17.LED Lights
[剩余数量：150] - [当前时间：20:52:52]

[状态：ok，尝试次数：1] - 1.Lighting > 12.Headlight Retainer
[剩余数量：149] - [当前时间：20:52:53]

[状态：ok，尝试次数：1] - 1.Lighting > 10.Lighting
[剩余数量：148] - [当前时间：20:52:54]

[状态：ok，尝试次数：1] - 2.Bumpers > 8.Bumper End
[剩余数量：147] - [当前时间：20:52:56]

[状态：ok，尝试次数：1] - 1.Lighting > 15.Wiring Harness
[剩余数量：146] - [当前时间：20:52:56]

[状态：ok，尝试次数：1] - 1.Lighting > 16.Headlight Adjust Screw
[剩余数量：145] - [当前时间：20:52:56]

[状态：ok，尝试次数：1] - 1.Lighting > 9.Tail Light Lens
[剩余数量：144] - [当前时间：20:52:56]

[状态：ok，尝试次数：1] - 1.Lighting > 13.Headlight Filler
[剩余数量：143] - [当前时间：20:52:58]

[状态：ok，尝试次数：1] - 2.Bumpers > 4.Bumper Filler
[剩余数量：142] - [当前时间：20:53:01]

[状态：ok，尝试次数：1] - 1.Lighting > 18.Third Brake Light
[剩余数量：141] - [当前时间：20:53:02]

[状态：ok，尝试次数：1] - 1.Lighting > 11.Back-Up Light Assembly
[剩余数量：140] - [当前时间：20:53:03]

[状态：ok，尝试次数：1] - 1.Lighting > 7.Park Light Assembly
[剩余数量：139] - [当前时间：20:53:04]

[状态：ok，尝试次数：1] - 1.Lighting > 5.Turn Signal A